# Import packages

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report
from sklearn.metrics import classification_report, confusion_matrix
%matplotlib inline

ModuleNotFoundError: No module named 'tensorflow'

# Import data

In [ ]:
smoke=pd.read_csv("/smoke.csv")

: 

# Columns explanation

#### Air Temperature
#### Air Humidity
#### TVOC: Total Volatile Organic Compounds; measured in parts per billion (Source)
#### eCO2: co2 equivalent concentration; calculated from different values like TVCO
#### Raw H2: raw molecular hydrogen; not compensated (Bias, temperature, etc.)
#### Raw Ethanol: raw ethanol gas (Source)
#### Air Pressure
#### PM 1.0 and PM 2.5: particulate matter size < 1.0 µm (PM1.0). 1.0 µm < 2.5 µm (PM2.5)
#### Fire Alarm: ground truth is "1" if a fire is there
#### CNT: Sample counter
#### UTC: Timestamp UTC seconds
#### NC0.5/NC1.0 and NC2.5: Number concentration of particulate matter. This differs from PM because NC gives the actual number of particles in the air. The raw NC is also classified by the particle size: < 0.5 µm (NC0.5); 0.5 µm < 1.0 µm (NC1.0); 1.0 µm < 2.5 µm (NC2.5)

In [ ]:
smoke

: 

In [ ]:
smoke.describe().transpose()

: 

In [ ]:
smoke.corr()

: 

In [ ]:
smoke.corr()['Fire Alarm'].sort_values()

: 

In [ ]:
smoke.corr()['Fire Alarm'].sort_values().plot(kind='bar')

: 

In [ ]:
cols=['Unnamed: 0' , 'CNT' , 'Raw Ethanol', 'Pressure[hPa]', 'UTC' ,'Humidity[%]' , 'PM1.0', 'PM1.0' , 'NC0.5' ,'NC1.0' ,'NC2.5']

: 

In [ ]:
smoke.drop(cols, axis=1, inplace=True) 

: 

In [ ]:
smoke.shape

: 

In [ ]:
smoke["Fire Alarm"].value_counts()

: 

In [ ]:
smoke.info()

: 

# Null values detection

In [ ]:
smoke.isnull().sum()

: 

# Outliers detection

In [ ]:
smoke.plot(kind='box', subplots=True, layout=(8,5), figsize=(17,20))

: 

In [ ]:
col = smoke.columns

: 

In [ ]:
def remove_outlier(col):
    sorted(col)
    Q1,Q3=np.percentile(col,[25,75])
    IQR=Q3-Q1
    lower_range= Q1-(1.5 * IQR)
    upper_range= Q3+(1.5 * IQR)
    return lower_range, upper_range

: 

In [ ]:
smoke_df=smoke.copy()
df0=smoke[smoke['Fire Alarm']==0.0]
df1=smoke[smoke['Fire Alarm']==1.0]

: 

In [ ]:
dec={"Colume":[],"outliers":[],"u_range":[] , 'l_range':[] ,"upper":[] , 'lower':[] ,
     "Nu_range":[] , 'Nl_range':[] ,"N_upper":[] , 'N_lower':[] 
    }
for column in smoke.iloc[:,:-1].columns:
    lr,ur=remove_outlier(df0[column])
    u_data=(smoke_df[(smoke_df['Fire Alarm']==0)&(smoke_df[column] > ur)])[column]
    l_data=(moke_df[(smoke_df['Fire Alarm']==0)&(smoke_df[column] < lr)])[column]
    dec['Colume'].append(column + " 0")
    dec['outliers'].append(len(u_data)+len(l_data))
    dec['upper'].append(len(u_data))
    dec['lower'].append(len(l_data))
    dec['u_range'].append(ur)
    dec['l_range'].append(lr)
    
    if not u_data.empty:
        u_data=sorted(u_data)
        index=int(round(len(u_data)*0.7,0))
        ur=u_data[index]
        dec['N_upper'].append(len(u_data[index:]))
    else:
        dec['N_upper'].append(len(u_data))
        
    if not l_data.empty:
        l_data=sorted(l_data)
        index=int(round(len(l_data)*0.3,0))
        lr=l_data[index]
        dec['N_lower'].append(len(l_data[:index]))
    else:
        dec['N_lower'].append(len(l_data))
        
    index=(smoke_df[(smoke_df['Fire Alarm']==0)&((smoke_df[column] < lr)|(smoke_df[column] > ur))]).index
    index=index.to_list()
    
    dec['Nu_range'].append(ur)
    dec['Nl_range'].append(lr)
    
    smoke_df.drop(index,inplace=True)
    
    lr,ur=remove_outlier(df1[column])
    u_data=(smoke_df[(smoke_df['Fire Alarm']==1)&(smoke_df[column] > ur)])[column]
    l_data=(smoke_df[(smoke_df['Fire Alarm']==1)&(smoke_df[column] < lr)])[column]
    dec['Colume'].append(column + " 1")
    dec['outliers'].append(len(u_data)+len(l_data))
    dec['upper'].append(len(u_data))
    dec['lower'].append(len(l_data))
    dec['u_range'].append(ur)
    dec['l_range'].append(lr)
    
    if not u_data.empty:
        u_data=sorted(u_data)
        index=int(round(len(u_data)*0.7,0))
        ur=u_data[index]
        dec['N_upper'].append(len(u_data[index:]))
    else:
        dec['N_upper'].append(len(u_data))
        
    if not l_data.empty:
        l_data=sorted(l_data)
        index=int(round(len(l_data)*0.3,0))
        lr=l_data[index]
        dec['N_lower'].append(len(l_data[:index]))
    else:
        dec['N_lower'].append(len(l_data))
        
        
    index=(smoke_df[(smoke_df['Fire Alarm']==1)&((smoke_df[column] < lr)|(smoke_df[column] > ur))]).index
    index=index.to_list()
    
    dec['Nu_range'].append(ur)
    dec['Nl_range'].append(lr)
    
    smoke_df.drop(index,inplace=True)
    

: 

In [ ]:
data_df.shape

: 

# EDA

In [ ]:
sns.countplot(x='Fire Alarm',data=smoke)

: 

In [ ]:
cols = list(smoke_df.columns)
cols_df=cols[1:]
plt.figure(figsize=(10,30))
for i in range(len(cols_df)):
    plt.subplot(8,3,i+1)
    plt.title(cols_df[i])
    plt.xticks(rotation=45)
    plt.hist(smoke[cols_df[i]])
    
plt.tight_layout()

: 

In [ ]:
cols = list(smoke_df.columns)
cols_df=cols[1:]
plt.figure(figsize=(10,30))
for i in range(len(cols_df)):
    plt.subplot(8,3,i+1)
    plt.title(cols_df[i])
    plt.xticks(rotation=45)
    plt.hist(smoke[cols_df[i]])
    
plt.tight_layout()

: 

In [ ]:
sns.jointplot(x="PM2.5", y="Fire Alarm",  data=smoke_df, height=6, ratio=2, color="yellow")

: 

In [ ]:
sns.jointplot(x="Raw H2", y="Fire Alarm",  data=data_df, height=6, ratio=2, color="blue")

: 

In [ ]:
sns.jointplot(x="eCO2[ppm]", y="Fire Alarm",  data=smoke_df, height=6, ratio=2, color="lime")
#Temperature[C]

: 

In [ ]:
sns.jointplot(x="TVOC[ppb]", y="Fire Alarm",  data=smoke_df, height=6, ratio=2, color="lime")

: 

In [ ]:
plt.scatter(smoke_df["Raw H2"], smoke_df["Fire Alarm"])

: 

In [ ]:
plt.scatter(smoke_df["eCO2[ppm]"], smoke_df["Fire Alarm"])

: 

In [ ]:
plt.scatter(smoke_df["TVOC[ppb]"], smoke_df["Fire Alarm"])

: 

In [ ]:
plt.scatter(smoke_df["PM2.5"], smoke_df["Fire Alarm"])

: 

# Split the data

In [ ]:
x = smoke_df.drop("Fire Alarm", axis = 1).values
y = smoke_df['Fire Alarm'].values 
x2 = smoke.drop("Fire Alarm", axis = 1).values
y2 = smoke['Fire Alarm'].values 

: 

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(x,y,test_size=0.2,random_state=0)
X2_train,X2_test,Y2_train,Y2_test = train_test_split(x2,y2,test_size=0.2,random_state=0)

: 

# Scale the data

In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
X2_train = sc.fit_transform(X_train)
X2_test = sc.transform(X_test)

: 

# Initializing Artificial Neural Network


In [ ]:
ann = tf.keras.models.Sequential()

: 

# Creating Hidden Layers

In [ ]:
ann.add(tf.keras.layers.Dense(units=6,activation="relu"))

: 

# Creating Output Layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=1,activation="sigmoid"))

: 

 # Creating Output Layer

In [ ]:
ann.compile(optimizer="adam",loss="binary_crossentropy",metrics=['accuracy'])

: 

# Fitting Artificial Neural Network

In [ ]:
history=ann.fit(x=X_train,y=Y_train,validation_data=(X_test,Y_test),batch_size=20,epochs=15)
history2=ann.fit(x=X2_train,y=Y2_train,validation_data=(X2_test,Y2_test),batch_size=20,epochs=15)

: 

In [ ]:
y_pred=ann.predict(X_test)
y2_pred=ann.predict(X2_test)

: 

In [ ]:
y_pred
y2_pred

: 

In [ ]:
type(y_pred)
type(y2_pred)

: 

In [ ]:
y_pred=list(y_pred)
y2_pred=list(y2_pred)

: 

In [ ]:
for i in range(len(y_pred)):
    if y_pred[i] >= .5:
        y_pred[i]=1
    else : y_pred[i]=0

for i in range(len(y2_pred)):
    if y2_pred[i] >= .5:
        y2_pred[i]=1
    else : y2_pred[i]=0

: 

# Evaluate Artificial Neural Network

In [ ]:
ann.evaluate(X_test,Y_test)
ann.evaluate(X2_test,Y2_test)

: 

In [ ]:
history.history
history2.history2

: 

In [ ]:
plt.title('Loss / Mean Squared Error w/out Outliers')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

plt.title('Loss / Mean Squared Error w/ Outliers')
plt.plot(history2.history2['loss'], label='train')
plt.plot(history2.history2['val_loss'], label='test')
plt.legend()
plt.show()

: 

# Classification Report

In [ ]:
print(confusion_matrix(Y_test,y_pred))
print(confusion_matrix(Y2_test,y2_pred))

: 

In [ ]:

print(classification_report(Y_test, y_pred, target_names=["Alarm", "No Alarm"]))
print(classification_report(Y2_test, y2_pred, target_names=["Alarm", "No Alarm"]))


: 

: 